Accompagne le script "Transcodage_from_file_to_bdd" 

Necessite d'avoir des fichiers dans "stockage_plan_de_compte", via le script "script_scraping.py"


Objectif : aller chercher les fichiers dans "stockage_plan_de_compte" et de les insérer dans la bdd après une légère préparation. 

- Entrée : les fichiers xml dans /stockage_plan_de_compte/
- Sortie : Une table de transcodage dans la bdd avec une clef permettant la correspondance avec la table de lignes_budgets

Stratégie naïve : 
- On va appliquer la même stratégie que pour l'extraction des lignes budges, de façon unitaire
- Malgré la non standardisation des .xml, lxml devrait ne pas produire les "@V" 
- Peu de modifications seront nécessaire, si ce n'est une colonne année et une colonne nomenclature issues du nom du fichier



Les dossiers ainsi que les COLONNES_TRANSCODAGE sont considérés comme des variables globales pour facilement les modifiers sans aller fouiller

A terme il faudra passer par duckdb et pas sqlite3

In [2]:
from lxml import etree
import pandas as pd 
import os
import sqlite3
import glob 

DOSSIER_PARENT = "."
BDD = './bdd_actes_budgetaires.db'
DOSSIER_TRANSCODAGE = "./csv_transcodage/"
DOSSIER_FICHIERS_TRANSCO = "./stockage_plan_de_compte/"
CHEMIN_PLAN_DE_COMPTE = glob.glob(os.path.join(DOSSIER_FICHIERS_TRANSCO, "*.xml"))
COLONNES_TRANSCODAGE = ['Nomenclature', 'Annee', 'Categorie', 'Code',       
                        'Lib_court', 'Libelle', 'PourEtatSeul', 
                        'Section', 'Special', 'TypeChapitre', 
                        'DEquip', 'REquip', 'DOES', 'DOIS', 
                        'DR', 'ROES', 'ROIS', 'RR', 
                        'RegrTotalise', 'Supprime', 'SupprimeDepuis']

Le parsing et les transformations se feront avec Lxml plutôt que xmltodict sur les lignes budgets.

- Avantage : Permet de récupérer des lignes non reconnnues comme étant des enfants par xmltodict, ce qui est le cas de la grande majorité des données
- Inconvénient : Incohérence entre les libs, va à terme créer de la dette, cependant, ce script n'est censé servir qu'une fois par an, quand de nouvelles nomenclatures sont ajoutées

In [4]:
def parsing_fichier(chemin) : 
 with open(chemin, "rb") as fichier_ouvert:
  arbre = etree.parse(fichier_ouvert)
  racine = arbre.getroot()
  enfants = racine.getchildren()
 return enfants

La vérification se fait de façon unitaire, ce serait plus rapide sous forme de liste comprehension mais la stratégie deviendrait mi-global, mi-unitaire

In [6]:
def isolement_plan_de_compte(plan_de_compte) :
 plan_de_compte_propre = plan_de_compte.replace("\\", '/').split('/')[2].split('.')[0]
 return plan_de_compte_propre

def fusionner_annee_nomenclature(conn = 'connect') -> list:
 ''' Permet de récupérer l'année et la '''
 cursor = conn.cursor()
 cursor.execute('''SELECT DISTINCT Annee || '-' || Nomenclature 
                      FROM Transcodage''')
 annee_nomenclature = [x[0] for x in cursor.fetchall()]
 return annee_nomenclature

Les fichiers récupérés via le script de scraping sont sous la forme "ANNEE_NOMENCLATURE.xml", permet de récupérer les élements qui seront injectés dans les colonnes correpondantes

In [ ]:
def extraction_metadonnees(chemin) : 
 chemin = chemin.replace("\\", '/')
 annee = chemin.split('/')[2].split('-')[0]
 nomenclature = chemin.split('/')[2].split('-', 1)[1].split('.')[0]
 return annee, nomenclature


Pour ne pas passer par les nombreux sous enfants, l'extraction va chercher toutes les lignes dans les sous ensemble NATURE et NATURE COMPTE.

Pour ça, on va chercher tout ce qui comporte @Code, ensuite les lignes affiliées sont converties en df avec une colonne permettant de spécifier le type de code

Deux df sont retournés car un pd.concat sera déjà fait lors de l'assemblage, on peut se questionner sur l'écart de vitesse entre l'appeller une fois ou deux. 

In [5]:

def extraction_nature(enfants) -> pd.DataFrame: 
 ''' Permet de récupérer les lignes de la branche Nature ( Nature et ContNat )'''
 nature_chapitre = enfants[0].getchildren()[0].xpath(".//*[@Code]")
 nature_compte = enfants[0].getchildren()[1].xpath(".//*[@Code]")
 liste_nature_chapitre = []
 liste_nature_compte = []

 for i in nature_chapitre :
  liste_nature_chapitre.append(i.attrib)
 df_nature_chapitre = pd.DataFrame(liste_nature_chapitre)
 df_nature_chapitre['Categorie'] = 'Nature'

 for i in nature_compte : 
  liste_nature_compte.append(i.attrib)
 df_nature_compte = pd.DataFrame(liste_nature_compte)
 df_nature_compte['Categorie'] = 'Nature_compte'

 return df_nature_chapitre, df_nature_compte


Même fonctionnement que pour extraction_nature. Ce sont cependant deux fonctions différentes pour ne pas que ce soit trop lourd et permettre de corriger d'éventuels problèmes spécifiques aux sous ensemble Fonction, Fonction_Compte et Fonction_Referentielle. 

Ex : Dans plusieurs nomenclatures Fonction & Fonction_Compte sont parfois entièrement vides même dans des collectivités SUP_3500, ça ne pose cependant aucune difficulté ici, même si ça a provoqué une panique lors de l'exploration et de la découverte de cette absence

In [ ]:
def extraction_fonction(enfants) -> pd.DataFrame:
 ''' Permet de récupérer les lignes de la branche Fonction ( Fonction et Fonction Compte et Fonction ref, ret, machin )'''
 fonction_chapitre = enfants[1].getchildren()[0].xpath(".//*[@Code]")
 fonction_compte = enfants[1].getchildren()[1].xpath(".//*[@Code]")
 fonction_ref = enfants[1].getchildren()[2].xpath(".//*[@Code]")

 liste_fonction_chapitre = []
 liste_fonction_compte = []
 liste_fonction_ref = []

 for i in fonction_chapitre:
    liste_fonction_chapitre.append(i.attrib)
 df_fonction_chapitre = pd.DataFrame(liste_fonction_chapitre)
 df_fonction_chapitre['Categorie'] = 'Fonction'

 for i in fonction_compte:
    liste_fonction_compte.append(i.attrib)
 df_fonction_compte = pd.DataFrame(liste_fonction_compte)
 df_fonction_compte['Categorie'] = 'Fonction_compte'

 for i in fonction_ref:
    liste_fonction_ref.append(i.attrib)
 df_fonction_ref = pd.DataFrame(liste_fonction_ref)
 df_fonction_ref['Categorie'] = 'Fonction_Ref'

 return df_fonction_chapitre, df_fonction_compte, df_fonction_ref


Note sur l'assemblage : 
- La liste COLONNES_TRANSCODAGE correspond au schéma de la table dans la bdd, elles doivent correpondre mais peuvent rapidement être modifié en cas de besoin
- Un df vide est crée, ses colonnes sont celles de COLONNES_TRANSCODAGE pour bien correspondre au schéma de la bdd
- La fonction creation_df_standard va recuperer les données du df initial est les insérer dans le df vide si les colonnes correspondent entre elles. 
- Ca nous permet d'éviter un drop columns qui peut provoquer des erreurs si les colonnes à supprimées ne sont pas présentes

In [ ]:
def creation_df_standard(df_nature_chapitre, df_nature_compte, 
                       df_fonction_chapitre, df_fonction_compte,
                       df_fonction_ref, annee, nomenclature) -> pd.DataFrame : 
 ''' Assemble les différents df, ajoute annee et nomenclature et standardise le schema de col pour
 qu'il corresponde à celui de la bdd'''
 df_assemblage = pd.concat([df_nature_chapitre, df_nature_compte,
                            df_fonction_chapitre, df_fonction_compte,
                            df_fonction_ref], ignore_index= True)
 df_assemblage['Nomenclature'] = nomenclature
 df_assemblage['Annee'] = annee
 df_transco = pd.DataFrame(columns= COLONNES_TRANSCODAGE)

 for i in df_assemblage.columns : #Ne conserve que les colonnes qui nous intéressent,
  if i in COLONNES_TRANSCODAGE :
   df_transco[i] = df_assemblage[i]
 return df_transco

In [ ]:
def insertion_dans_bdd(df_transco, conn = 'connect') : 
 """ Insert dans une bdd les données maintenant transformées et en sort un csv à jour """
 df_transco.to_sql('Transcodage', conn,
                    if_exists='append', index=False)

Pour chaque élement dans le dossier stockage_plan_de_compte le script va ainsi : 
 1. Ouvrir une connection à la bdd
 2. Vérifier si le plan de compte est déjà dans la bdd
 3. Si c'est le cas, alors on passe au plan de compte suivant
 4. Si ce n'est pas le cas, on extrait et insert ses informations
 5. Dans les deux situations, avant de passer au plan de compte suivant, on ferme la connection

In [ ]:
def main() : 
 for plan_de_compte in CHEMIN_PLAN_DE_COMPTE :
  try :
   connect = sqlite3.connect(BDD)
   if isolement_plan_de_compte(plan_de_compte) not in fusionner_annee_nomenclature(connect) : 
    enfant = parsing_fichier(plan_de_compte)
    annee, nomenclature = extraction_metadonnees(plan_de_compte)
    df_nature1, df_nature2 = extraction_nature(enfant)
    df_f1, df_f2, df_f3 =  extraction_fonction(enfant)
    df_test = creation_df_standard(df_nature1, df_nature2, df_f1, df_f2, df_f3, annee, nomenclature)
    insertion_dans_bdd(df_test, connect)
   else : 
    pass 
  finally : 
   connect.commit()
   connect.close()